In [1]:
import pandas as pd
import numpy as np

In [8]:
pd.set_option("display.max_columns", None)

In [2]:
X_features= ["ncodpers",
 'ind_empleado','pais_residencia','sexo','age','ind_nuevo','antiguedad','indrel',
 'indrel_1mes','tiprel_1mes','indresi','indext','canal_entrada','indfall',
 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento']

notuse = ["ult_fec_cli_1t","nomprov", "conyuemp", "fecha_dato", "fecha_alta"]

y_features = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

train_cols = X_features + y_features

In [3]:
df_train = pd.read_csv("all_clean.csv", usecols = train_cols)

In [185]:
df_test = pd.read_csv("test_by_wk.csv")

C:\Users\ledes\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### train

In [ ]:
df_train = df_train[df_train['ind_nuevo'] == 0]
df_train = df_train[df_train['antiguedad'] != -999999]
df_train = df_train[df_train['indrel'] == 1]
df_train = df_train[df_train['indresi'] == 'S']
df_train = df_train[df_train['indfall'] == 'N']
df_train = df_train[df_train['tipodom'] == 1]
df_train = df_train[df_train['ind_empleado'] == 'N']
df_train = df_train[df_train['pais_residencia'] == 'ES']
df_train = df_train[df_train['indrel_1mes'] == 1]
df_train = df_train[df_train['tiprel_1mes'] == ('A' or 'I')]
df_train = df_train[df_train['indext'] == 'N']

In [7]:
df_train.to_csv("1st_preprocessing.csv", index = False)

In [39]:
df_train = pd.read_csv("1st_preprocessing.csv")

In [40]:
# null - 최빈값
df_train["canal_entrada"] = df_train["canal_entrada"].fillna(df_train["canal_entrada"].value_counts().index[0])
# null - 최빈값
df_train["segmento"] = df_train["segmento"].fillna(df_train["segmento"].value_counts().index[0])
# null - ffill
df_train["sexo"] = df_train["sexo"].fillna(method = "ffill")

In [41]:
#segmento - dtype 변형
def func_for_segmento(x) :    
    if type(x) != float :
        if "1" in x :
            result = 1
        elif "2" in x :
            result = 2
        elif "3" in x :
            result = 3
    else : 
        result = None
    return result

df_train["segmento"] = df_train["segmento"].apply(func_for_segmento)

#canal_entrada - 상위 4개 제외, 그 이외 vlaue는 5번째껄로 통일
target_values = ["KHE", "KAT", "KFC", "KHQ"]

df_train["canal_entrada"] = df_train["canal_entrada"].apply(lambda x : "KHM" if x not in target_values else x)

In [42]:
#각 지역의 중앙값으로 대체
df_train["renta"]   = pd.to_numeric(df_train["renta"], errors="coerce")
df_test["renta"]   = pd.to_numeric(df_test["renta"], errors="coerce")

unique_prov = df_test[df_test.cod_prov.notnull()].cod_prov.unique()
grouped = df_test.groupby("cod_prov")["renta"].median()

for cod in unique_prov:
    df_test.loc[df_test['cod_prov']==cod,['renta']] = df_test.loc[df_test['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values
    df_train.loc[df_train['cod_prov']==cod,['renta']] = df_train.loc[df_train['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values

In [89]:
df_train.loc[df_train["ncodpers"] == 1234433, ["cod_prov"]] = df_train["cod_prov"].value_counts().index[0]
df_train.loc[df_train["ncodpers"] == 1234433, ["renta"]] = df_train.loc[df_train["ncodpers"] == 1234433, ["renta"]].renta.median()

In [156]:
#종속변수 null - 0으로 대체
#loc에 boolean사용시, 내가 사용하는 기준이 row인지, columns 인지 분별할 것
df_train.loc[df_train["ind_nomina_ult1"].isnull(), ["ind_nomina_ult1"]] = 0
df_train.loc[df_train["ind_nom_pens_ult1"].isnull(), ["ind_nom_pens_ult1"]] = 0

In [159]:
df_train.isnull().sum()

ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
ind_nuevo                0
antiguedad               0
indrel                   0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
canal_entrada            0
indfall                  0
tipodom                  0
cod_prov                 0
ind_actividad_cliente    0
renta                    0
segmento                 0
ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
ind_reca_fin_ult1        0
i

In [161]:
df_train.head()

,ncodpers,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1377105,N,ES,V,36.0,0.0,16.0,1.0,1.0,A,S,N,KHM,N,1.0,1.0,1.0,116497.515,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0
1,760289,N,ES,V,46.0,0.0,88.0,1.0,1.0,A,S,N,KFC,N,1.0,1.0,1.0,116497.515,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,1150802,N,ES,H,22.0,0.0,32.0,1.0,1.0,A,S,N,KHE,N,1.0,1.0,1.0,116497.515,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,179011,N,ES,V,41.0,0.0,187.0,1.0,1.0,A,S,N,KAT,N,1.0,1.0,1.0,116497.515,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,1
4,761473,N,ES,H,45.0,0.0,88.0,1.0,1.0,A,S,N,KAT,N,1.0,1.0,1.0,116497.515,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [162]:
df_train.to_csv("final_preprocessing.csv", index = False)

### test

In [186]:
df_test = df_test[df_test['ind_nuevo'] == 0]
df_test = df_test[df_test['antiguedad'] != -999999]
df_test = df_test[df_test['indrel'] == 1]
df_test = df_test[df_test['indresi'] == 'S']
df_test = df_test[df_test['indfall'] == 'N']
df_test = df_test[df_test['tipodom'] == 1]
df_test = df_test[df_test['ind_empleado'] == 'N']
df_test = df_test[df_test['pais_residencia'] == 'ES']
df_test = df_test[df_test['indrel_1mes'] == 1]
df_test = df_test[df_test['tiprel_1mes'] == ('A' or 'I')]
df_test = df_test[df_test['indext'] == 'N']

In [188]:
#각 지역의 중앙값으로 대체
df_test["renta"]   = pd.to_numeric(df_test["renta"], errors="coerce")

unique_prov = df_test[df_test.cod_prov.notnull()].cod_prov.unique()
grouped = df_test.groupby("cod_prov")["renta"].median()

for cod in unique_prov:
    df_test.loc[df_test['cod_prov']==cod,['renta']] = df_test.loc[df_test['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values

In [189]:
# null - 최빈값
df_test["canal_entrada"] = df_test["canal_entrada"].fillna(df_test["canal_entrada"].value_counts().index[0])
# null - 최빈값
df_test["segmento"] = df_test["segmento"].fillna(df_test["segmento"].value_counts().index[0])

In [190]:
#canal_entrada - 상위 4개 제외, 그 이외 vlaue는 5번째껄로 통일
target_values = ["KHE", "KAT", "KFC", "KHQ"]

df_test["canal_entrada"] = df_test["canal_entrada"].apply(lambda x : "KHM" if x not in target_values else x)

In [191]:
df_test["segmento"] = df_test["segmento"].fillna(df_test["segmento"].value_counts().index[0])

def func_for_segmento(x) :    
    if type(x) != float :
        if "1" in x :
            result = 1
        elif "2" in x :
            result = 2
        elif "3" in x :
            result = 3
    else : 
        result = None
    return result

df_test["segmento"] = df_test["segmento"].apply(func_for_segmento)

In [196]:
df_test.reset_index(inplace = True, drop = True)

In [201]:
df_test = df_test.drop(columns = "index")

In [202]:
df_test.to_csv("test_preprocessing.csv", index = False)